## 2. Feature_Extraction_from_VGG16_to_InceptionV3

### References:
1. https://github.com/ypwhs/dogs_vs_cats
2. https://www.kaggle.com/yangpeiwen/keras-inception-xception-0-47

### Import pkgs

In [1]:
import os, time, math, h5py, pickle

from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.preprocessing.image import *
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19
from keras.applications.inception_v3 import preprocess_input as preprocess_input_inception_v3

Using TensorFlow backend.


In [2]:
def pickle_dump(data, file):
    with open(file, 'wb') as f:
        pickle.dump(data, f)

def pickle_load(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
    return data

a = list(range(10))
print(a)
demo_file = os.path.join(os.getcwd(), 'temp', 'pickle_demo.pkl')
print(demo_file)
pickle_dump(a, demo_file)
new_a = pickle_load(demo_file)
print(new_a)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
D:\Kaggle\dog-breed-identification\temp\pickle_demo.pkl
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [3]:
def get_features(MODEL, model_name, image_size, folder_name, batch_size=1, lambda_func=None):
    print('{0} start.'.format(model_name))
    start_time = time.time()
    
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        print(lambda_func.__name__)
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    cwd = os.getcwd()
    data_train_path = os.path.join(cwd, 'input', folder_name)
    
    gen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)
    train_generator = gen.flow_from_directory(
        data_train_path, 
        image_size, 
        shuffle=False, 
        batch_size=batch_size
    )
#     val_generator  = gen.flow_from_directory(data_val_path,  image_size, shuffle=False, 
#                                               batch_size=batch_size)
#     test_generator  = gen.flow_from_directory(data_test_path,  image_size, shuffle=False, 
#                                               batch_size=batch_size)
    train_steps = 63 #math.ceil(len(train_generator.classes)/batch_size)
    print('train_steps:', train_steps)
#     train = model.predict_generator(train_generator, verbose=1, steps=len(train_generator.classes))
    train = model.predict_generator(train_generator, verbose=1, steps=train_steps)
#     val = model.predict_generator(val_generator, verbose=1, steps=len(val_generator.classes))
#     test = model.predict_generator(test_generator, verbose=1, steps=10357)
#     train = model.predict_generator(train_generator, verbose=1, steps=10)
#     val = model.predict_generator(val_generator, verbose=1, steps=10)
#     test = model.predict_generator(test_generator, verbose=1, steps=10)
#     print(test_generator.filenames)
    
    class_indices_file = os.path.join(cwd, 'feature', 'feature_{0}_{1}_{2}_class_indices.pkl'.format(model_name, folder_name, 171023))
    if os.path.exists(class_indices_file):
        os.remove(class_indices_file)
    print(class_indices_file)
    pickle_dump(train_generator.class_indices, class_indices_file)

    h5py_file_name = os.path.join(cwd, 'feature', 'feature_{0}_{1}_{2}.h5'.format(model_name, folder_name, 171023))
    print(h5py_file_name)
    if os.path.exists(h5py_file_name):
        os.remove(h5py_file_name)
    with h5py.File(h5py_file_name) as h:
        h.create_dataset('x_%s' % folder_name, data=train)
        h.create_dataset('classes_%s' % folder_name, data=train_generator.classes)
#         h.create_dataset("val", data=val)
#         h.create_dataset("val_label", data=val_generator.classes)
#         h.create_dataset("test", data=test)

    print(train.shape)
    print(len(train_generator.classes))
#     print(val.shape)
#     print(len(val_generator.classes))
#     print(test.shape)
    
    print(dir(train_generator))
#     print(train_generator.filenames)
    print(train_generator.samples)
    print(len(train_generator.classes))
    print(len(train_generator.class_indices))
    print(train_generator.num_classes)
    print(len(train_generator.image_shape))
    print(train_generator.batch_index)
#     print(len(list(train_generator.index_generator)))
    print(len(train_generator.index_array))
    print(type(train_generator.class_indices))
    end_time = time.time()
    print('Spend time: {0} s'.format(end_time-start_time))
    
    return train_generator
    

In [4]:
%pdb off

Automatic pdb calling has been turned OFF


In [5]:
# get_features(VGG16, 'VGG16', (224, 224), 'data_train', 1)
train_generator = get_features(VGG16, 'VGG16', (224, 224), 'data_val', 17, preprocess_input_vgg16)
# get_features(VGG16, (224, 224), 'data_test', 1)

VGG16 start.
wrapper
Instructions for updating:
Colocations handled automatically by placer.
Found 1023 images belonging to 120 classes.
train_steps: 63
63/63 [==============================] - 665s 11s/step
D:\Kaggle\dog-breed-identification\feature\feature_VGG16_data_val_171023_class_indices.pkl
D:\Kaggle\dog-breed-identification\feature\feature_VGG16_data_val_171023.h5
(1057, 512)
1023
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_flow_index', '_get_batches_of_transformed_samples', '_set_index_array', '_tf_api_names', '_tf_api_names_v1', 'allowed_class_modes', 'batch_index', 'batch_size', 'class_indices', 'class_mode', 'classes', 'color_m

In [6]:
print(len(train_generator.class_indices))
print(len(train_generator.classes))
print(len(train_generator.index_array))

120
1023
1023


In [7]:
demo_file = os.path.join(os.getcwd(), 'feature', 'feature_VGG16_data_val_171023_class_indices.pkl')
print(demo_file)
# pickle_dump(a, demo_file)
new_a = pickle_load(demo_file)
print(new_a)

D:\Kaggle\dog-breed-identification\feature\feature_VGG16_data_val_171023_class_indices.pkl
{'affenpinscher': 0, 'afghan_hound': 1, 'african_hunting_dog': 2, 'airedale': 3, 'american_staffordshire_terrier': 4, 'appenzeller': 5, 'australian_terrier': 6, 'basenji': 7, 'basset': 8, 'beagle': 9, 'bedlington_terrier': 10, 'bernese_mountain_dog': 11, 'black-and-tan_coonhound': 12, 'blenheim_spaniel': 13, 'bloodhound': 14, 'bluetick': 15, 'border_collie': 16, 'border_terrier': 17, 'borzoi': 18, 'boston_bull': 19, 'bouvier_des_flandres': 20, 'boxer': 21, 'brabancon_griffon': 22, 'briard': 23, 'brittany_spaniel': 24, 'bull_mastiff': 25, 'cairn': 26, 'cardigan': 27, 'chesapeake_bay_retriever': 28, 'chihuahua': 29, 'chow': 30, 'clumber': 31, 'cocker_spaniel': 32, 'collie': 33, 'curly-coated_retriever': 34, 'dandie_dinmont': 35, 'dhole': 36, 'dingo': 37, 'doberman': 38, 'english_foxhound': 39, 'english_setter': 40, 'english_springer': 41, 'entlebucher': 42, 'eskimo_dog': 43, 'flat-coated_retriever'

In [8]:
# get_features(VGG19, 'VGG19', (224, 224), 'data_train', 1)
# get_features(VGG19, 'VGG19', (224, 224), 'data_val', 1)

In [9]:
# get_features(ResNet50, (224, 224), 1)

In [10]:
# get_features(Xception, (299, 299), 1, xception.preprocess_input)

In [11]:
# get_features(InceptionV3, (299, 299), 1, inception_v3.preprocess_input)

In [12]:
# get_features(InceptionResNetV2, (299, 299), 1, inception_v3.preprocess_input)

In [13]:
print('Done !')

Done !
